# Drive access
---

In [5]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os

g_login = GoogleAuth()
g_login.LocalWebserverAuth()
drive = GoogleDrive(g_login)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1003652371916-aljjqru0t2u99r3cnru9m69ujodst5bb.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [28]:
import xml.etree.ElementTree as ET
import pandas as pd

class kml2csv:
    """ Enter filename location from currdir
    ref : https://gist.github.com/bmcbride/9689580
    """
    def __init__(self,file_name):
        self.file_name = file_name
        self.csv = None
        self.root = None
    def isVid(self):
        if self.file_name[-7:-4] == "mp4":
            return True
        return False
    
    def __str__(self):
        """
        Display file name
        """
        return str(self.file_name)
    
    def extract_loc(self):
        ls = []
        if self.isVid():
            # location extraction from video
            for child in self.root.findall('./Document/Placemark/ExtendedData/SchemaData'):

                fid = child[0].text
                lat = child[1].text
                lon = child[2].text
                
                ls.append((int(fid),float(lat),float(lon)))

        else:
            # location extraction from image
            child = self.root.findall("./Document/Placemark/Point/coordinates")
            p = child[0].text.strip().split('\n')[0].split(",")
            print(p)
            ls.append((0,float(p[1]),float(p[0])))

        return pd.DataFrame(ls)  # cnvt to dataframe
                
    def run(self):
        tree = ET.parse(self.file_name)
        self.root = tree.getroot()
        self.csv = self.extract_loc()
        self.csv.columns = ["id","lat","lon"]
        return self.csv

/home/alvin/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/alvin/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [1]:
# uploading and processing file

loc = os.listdir("./Pictures")
for i in loc:
    if "kml" in i:
        kml_file = i
        
    if "mp4" in i:
        vid_name = i
        
        
        
# kml_file = vid_name+".kml"

kml2csv(kml_file).run().to_csv(vid_name+".csv",index = False)

file5 = drive.CreateFile({"parents": [{"kind": "drive#fileLink","id": '1t-vTle0eQF9T5_Hw3NkbGPIB7TV9Xpbt' }] ,
                                   'title':vid_name})
file5.SetContentFile(vid_name)
file5.Upload() # Upload the file.

file4 = drive.CreateFile({"parents": [{"kind": "drive#fileLink","id": '1t-vTle0eQF9T5_Hw3NkbGPIB7TV9Xpbt' }] ,
                                   'title':kml_file})

file4.SetContentFile(vid_name+".csv")
file4.Upload() # Upload the file.


print("Upload Successfull")

Upload Successfull
